New text preprocessing tricks
No tree generation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step1. install packages and mount drive

In [2]:
!pip install torch torchvision transformers

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 31.7 MB/s 
     |████████████████████████████████| 596 kB 51.6 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 880 kB 37.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7d4a40f3c948f88b571db8baceaa12ea5bab8f6df74c929a4e8bd9cec2a1cac4
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer, AutoModel

MODEL = "cardiffnlp/twitter-roberta-base-mar2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModel.from_pretrained(MODEL)


#model = AutoModelForSequenceClassification.from_pretrained("tristantristantristan/rumor")
##inputs = tokenizer("I love AutoTrain", return_tensors="pt")

#outputs = model(**inputs)
#outputs

Downloading:   0%|          | 0.00/346 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-mar2022 were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-mar2022 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [1]:
import json
import os
import datetime
import pandas as pd
from nltk.corpus import stopwords
from nltk import TweetTokenizer
import re
from transformers import BertTokenizer
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
import os
import copy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# read data for task1 training and dev set
def read_all_file(data_dir):
    file_name_list = os.listdir(data_dir)
    total_df = pd.DataFrame()
    for file in file_name_list:
        file_path = data_dir + '/' + file
        with open(file_path,'r') as f:
            small_df = convert_to_df(f.readline())
            total_df = total_df.append(small_df)
    return total_df


def convert_to_df(line):
    value_dict = json.loads(line)
    small_length = 0
    long_add = []
    for key,value in value_dict.items():
        rumor_id = key
        tweet_list = value
        for tweet_info in tweet_list:
            info_list = ['author_id','lang','text','created_at','in_reply_to_user_id','referenced_tweets','id']
            single_tweet = {'index':rumor_id}
            for info in info_list:
                single_tweet[info] = tweet_info.get(info,None)
            #referenced_tweet = tweet_info.get(, {})
            #single_tweet['type'] = referenced_tweet.get('type',None)
            #single_tweet['refer_id'] = referenced_tweet.get('id',None)
            long_add.append(single_tweet)
    small_length = pd.DataFrame(long_add)
#        small_length += len(tweet_list)
    return small_length

helper functions for reading labels

In [3]:
def read_label_from_txt(filename):
    result = {}
    with open(filename,'r') as f:
        for index, line in enumerate(f.readlines()):
            result[index] = line.strip('\n')

    return result

helper functions for data and text preprossing

In [4]:
def convert_str_time(line):
    try:
        timestamp = datetime.datetime.strptime(line, "%Y-%m-%dT%H:%M:%S.%fZ")
        return timestamp
    except:
        return None



# reference https://stackoverflow.com/questions/9760588/how-do-you-extract-a-url-from-a-string-using-python
def Find(string):
    # findall() has been used
    # with valid conditions for urls in string
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.search(regex, string)
    if url is not None:
        return True
    else:
        return False

def clean_twitter_text_result(line,tt):
    #stop = stopwords.words('english')
    #tt = TweetTokenizer()
    tokenized_line = tt.tokenize(line)
    #tokenized_line = tokenizer.tokenize(line)
    bag_of_words = []
    for word in tokenized_line:
        word = word.lower()
        if word.startswith("#"):
          bag_of_words = bag_of_words + break_hashtag(word)
        else:
          word = '@user' if word.startswith('@') and len(word) > 1 else word
        #word = word.lstrip('#') if word.startswith('#') and len(word) > 1 else word
          word = 'http' if word.startswith('http') or word.startswith('https') else word
        #if re.search(r'([a-z]+)', word) and not word in stop:
        #    if not Find(word):
          bag_of_words.append(word)
    new = ' '.join(bag_of_words)
    #tokenized_line = tokenizer.tokenize(new)
    #tokenized_line = ['[CLS]'] + tokenized_line + ['[SEP]']
    return new

def break_hashtag(word):
  word_list = tokenize_hashtags(word.lstrip('#'))
  return word_list

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching


def tokenize_hashtags(hashtag):
    ###
    # Your answer BEGINS HERE
    ###
    #tokenized = {}
        # define two cursors, the slow cursor indicates the start of unmatched string
        # the fast cursor traverses the remaining sting
        slow_cur, fast_cur = 0,1
        output = []
        while slow_cur < len(hashtag):
            matched_word = hashtag[slow_cur:fast_cur] # initialize matched_word
            while fast_cur <= len(hashtag):
                current_word = hashtag[slow_cur:fast_cur]
                # see if a longer word after lemmatization is found, update matched_word 
                if lemmatize(current_word) in words and len(current_word) > len(matched_word):
                    matched_word = current_word
                fast_cur += 1
            # reset the two cursor when a new word is matched and added to output
            slow_cur = slow_cur + len(matched_word)
            fast_cur = slow_cur + 1
            output.append(matched_word)
    #    tokenized[hashtag] = output
        return output


Test text preprocessing

In [8]:
tt = nltk.TweetTokenizer()
clean_twitter_text_result("@jonnychen_1132, It's been a long day, without you my friend. #Welovepaulwalker url:http://t.co/45hkhgdsjh",tt)

"@user , it's been a long day , without you my friend . we love paul walker url : http"

helper function to concact tweets in a propagation sequence

In [9]:
def concact_text(text:pd.Series):
    concact_result = []
    seg_ids = []
    i = 0
    for index, item in text.iteritems():
        concact_result = concact_result + item + ['[SEP]']
        seg_ids = seg_ids + [i] *( len(item) + 1)
        i += 1
    concact_result = ['[CLS]'] + concact_result
    seg_ids = [0] + seg_ids
    return concact_result, seg_ids

In [10]:
class Rumor_tree:

    def __init__(self, root,Graph):
        # default dictionary to store graph
        self.graph = Graph
        self.root = root
        self.all_path = []
        self.leaf_node = self.find_leaf_node()

    def find_leaf_node(self):
        tmp = []
        for item, value in self.graph.items():
            if len(value) == 0:
               tmp.append(item)
        return tmp

    def init_path(self):
        for leaf in self.leaf_node:
            self.printAllPaths(self.root, leaf)
        return self.all_path
    '''A recursive function to print all paths from 'u' to 'd'.
    visited[] keeps track of vertices in current path.
    path[] stores actual vertices and path_index is current
    index in path[]'''

    def printAllPathsUtil(self, u, d, visited, path):

        # Mark the current node as visited and store in path
        visited.append(u)
        #visited[u] = True
        path.append(u)

        # If current vertex is same as destination, then print
        # current path[]
        if u == d:
            self.all_path.append(copy.deepcopy(path))
            #print(path)
        else:
            # If current vertex is not destination
            # Recur for all the vertices adjacent to this vertex
            for i in self.graph[u]:
                if not i in visited:
                    self.printAllPathsUtil(i, d, visited, path)

        # Remove current vertex from path[] and mark it as unvisited
        path.pop()
        #visited[u] = False
        visited.remove(u)


    # Prints all paths from 's' to 'd'
    def printAllPaths(self, s, d):

        # Mark all the vertices as not visited
        #visited = [False] * len(self.graph.keys())
        visited = []
        # Create an array to store paths
        path = []

        # Call the recursive helper function to print all paths
        self.printAllPathsUtil(s, d, visited, path)

Build and concact rumour sequence

In [11]:
def build_rumor_tree(df:pd.DataFrame):
    # first try, return sequence using time
    rumor_graph = {}
    source_tweet = df.iloc[0]
    rumor_graph[source_tweet['id']] = []
    replys = df.iloc[1:]
    for index, tweet in replys.iterrows():
        tweet_id = tweet['id']

        reply_relation = tweet['referenced_tweets']
        if type(reply_relation) == list and len(reply_relation) > 0:

          parent_tweet_id = reply_relation[0]['id']
          rumor_graph[tweet_id] = []
          if parent_tweet_id in rumor_graph.keys():
            rumor_graph[parent_tweet_id].append(tweet_id)

    time_series_list = []
    #a = df.sort_values(by='created_at')

    return source_tweet['id'], rumor_graph

def build_rumor_tree_v11(df:pd.DataFrame):
    # first try, return sequence using time
    #df = df.sort_values(by='created_at')
    rumor_graph = {}
    source_tweet = df.iloc[0]
    rumor_graph[source_tweet['id']] = []
    replys = df.iloc[1:]
    for index, tweet in replys.iterrows():
        tweet_id = tweet['id']
        parent_tweet_id = tweet['in_reply_to_status_id']
        #reply_relation = tweet['referenced_tweets'][0]
        #parent_tweet_id = reply_relation
        rumor_graph[tweet_id] = []
        if parent_tweet_id in rumor_graph.keys():
            rumor_graph[parent_tweet_id].append(tweet_id)

    #time_series_list = []


    return source_tweet['id'], rumor_graph

    
def split_rumor_tree(source_id, rumor_tree):
    if source_id is None:
        return
    rumor = Rumor_tree(source_id,rumor_tree)
    total_path_set = rumor.init_path()

    return len(total_path_set),total_path_set

def concact_sentence(df:pd.DataFrame):
    long_sentence = ''
    for _, item in df.iterrows():
      #long_sentence = long_sentence +  item['text'] + ' [SEP] '
      long_sentence = long_sentence +  item['text'] + ' '
    return long_sentence


def sentence_paste(df:pd.DataFrame):
  long_sentence = []
  for _, item in df.iterrows():
    long_sentence.append(item['text'])
  return long_sentence


def make_sequence_data(df):
      groupped = df.groupby(['problem_num'])
      big_df = pd.DataFrame()
      for index, group in groupped:
        new_train_dict = []
        source_tweet_id, rumor_tree = build_rumor_tree(group)
        tree_number, tree_list = split_rumor_tree(source_tweet_id, rumor_tree)
        for split_tree in tree_list:
            split_rumor = group.loc[group['id'].isin(split_tree)]
            source_sentence = split_rumor.loc[split_rumor['id'] == source_tweet_id]
            other_sentence = split_rumor.loc[split_rumor['id'] != source_tweet_id]

            #other_sentence = split_rumor.iloc[1:,'text']
            single_instance_sentence = concact_sentence(other_sentence)
            #single_instance_sentence = concact_sentence(split_rumor)
            single_instance_problem_num = split_rumor['problem_num'].values[0]
            single_instance_label = split_rumor['label'].values[0]
            new_train_dict.append({'source_text': source_sentence['text'].values[0],
                                   'problem_num':single_instance_problem_num,
                                   'reply':single_instance_sentence,
                                   'label': single_instance_label})
        new_train_df = pd.DataFrame(new_train_dict)
        big_df = big_df.append(new_train_df)
      return big_df

def make_sequence_data_test(df):
    groupped = df.groupby(['problem_num'])
    big_df = pd.DataFrame()
    for index, group in groupped:
        new_train_dict = []
        
        source_tweet_id, rumor_tree = build_rumor_tree_v11(group)
        tree_number, tree_list = split_rumor_tree(source_tweet_id, rumor_tree)
        for split_tree in tree_list:
            split_rumor = group.loc[group['id'].isin(split_tree)]
            single_instance_sentence = concact_sentence(split_rumor)
            single_instance_problem_num = split_rumor['problem_num'].values[0]
            new_train_dict.append({'text': single_instance_sentence,
                                   'problem_num': single_instance_problem_num})
 
        new_train_df = pd.DataFrame(new_train_dict)
        big_df = big_df.append(new_train_df)
    return big_df


Build train and dev dataset

In [12]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class TweetDataset():

    def __init__(self, data_dir, maxlen,model_name):

        #Store the contents of the file in a pandas dataframe
        self.df = self.init_data(data_dir)

        #Initialize the BERT tokenizer
        #self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
        #self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
        #self.tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-2021-124m")
 #       self.tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-mar2022")

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.maxlen = maxlen

        self.tt = TweetTokenizer()
        self.stop = stopwords.words('english')


    def __len__(self):
        return len(self.df)


    def init_data(self,data_dir):
        data_dir = data_dir
        train_data = read_all_file(data_dir)
        type_of_data = data_dir.replace('/content/drive/MyDrive/NLP_data/t1_','')
        
        train_label = read_label_from_txt('/content/drive/MyDrive/NLP_data/project-data/'+ type_of_data + '.label.txt')
        #train_label = train_label.to_dict()
        train_data['label'] = train_data['index'].apply(lambda x: train_label[int(x)])
        to_number = {'nonrumour':0,'rumour':1}
        train_data['label'] = train_data['label'].map(to_number)

        train_data['created_at'] = train_data['created_at'].apply(lambda x: convert_str_time(x))
        #t1_train_org['text_clean'] = t1_train_org['text'].apply(lambda x: )
        train_data.rename(columns={'index': 'problem_num'}, inplace=True)
        train_data = make_sequence_data(train_data)
        train_data = train_data.reset_index()
        return train_data

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        source = self.df.loc[index, 'source_text']
        reply = self.df.loc[index,'reply']
        label = self.df.loc[index, 'label']
        source = clean_twitter_text_result(source, self.tt)
        reply = clean_twitter_text_result(reply, self.tt)


        inputs = self.tokenizer.encode_plus(
          source,
          reply,
          padding='max_length',
          add_special_tokens=True,
          return_attention_mask=True,
          max_length=self.maxlen,
          truncation='longest_first'
          )
        ids = inputs['input_ids']
        #token_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']

        tokens_ids_tensor = torch.tensor(ids,dtype=torch.long)
        attn_mask = torch.tensor(mask, dtype=torch.long)
        #token_type_ids = torch.tensor(token_ids, dtype=torch.long)
        #label = torch.tensor(label, dtype=torch.long)
        #Preprocessing the text to be suitable for BERT
        # the first function removes stopwords and urls using NLTK, it 
        # conflicts default bert tokenizer 
        #tokens = clean_twitter_text_result(sentence, self.tokenizer)
        #tokenized_line = tokenizer.tokenize(sentence)
     
       
        #tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        #tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        #attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label #, token_type_ids, label

In [13]:
def convert_str_time_v11(line):
    try:
        timestamp = datetime.datetime.strptime(line, "%a %b %d %H:%M:%S %z %Y")
        return timestamp
    except:
        return None



Build text data, which is different because text data are using twitter API v1.1 instead of v2

In [14]:
class TestDataset():

    def __init__(self, data_dir, maxlen,model_name):

        # Store the contents of the file in a pandas dataframe
        self.maxlen = maxlen
        self.file_dir = data_dir + '/tweet-objects/tweet-objects'
        self.file_list = os.listdir(self.file_dir)
        self.not_found = 0
        self.df = self.init_data(data_dir)

        # Initialize the BERT tokenizer
        #self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tt = TweetTokenizer()
        self.stop = stopwords.words('english')
    def check(self):
        return self.not_found

    def __len__(self):
        return len(self.df)

    def read_one_file(self, tweet_id):
        filename = tweet_id + '.json'
        if filename in self.file_list:
            file_path = self.file_dir + '/' + filename
            with open(file_path, 'r') as f:
                line = f.readline()
                line_json = json.loads(line)
                return line_json
        else:
            return None

    def init_data(self, data_dir):
        test_file_list_dir = '/content/drive/MyDrive/NLP_data/project-data/test.data.txt'
        file_reader = open(test_file_list_dir, 'r')
        long_add = []
        for index, line in enumerate(file_reader.readlines()):
            ids = line.strip('\n').split(',')
            for tweet_id in ids:
                single_line_result = self.read_one_file(tweet_id)
                if single_line_result:
                    info_list = ['id', 'author_id', 'lang', 'text', 'created_at', 'in_reply_to_status_id',
                                 'referenced_tweets']
                    single_tweet = {'problem_num': index}
                    for info in info_list:
                        single_tweet[info] = single_line_result.get(info, None)
                    single_tweet['in_reply_to_status_id'] = int(single_tweet['in_reply_to_status_id']) if single_line_result.get('in_reply_to_status_id', None) else None
                    long_add.append(single_tweet)
                else:
                    self.not_found += 1
        total_test_df = pd.DataFrame(long_add)
        # train_label = read_label_from_txt('/content/drive/MyDrive/NLP_data/project-data/test.label.txt')
        total_test_df['created_at'] = total_test_df['created_at'].apply(lambda x: convert_str_time_v11(x))
        #total_test_df.rename(columns={'index': 'problem_num'}, inplace=True)
        #total_test_df = total_test_df.loc[total_test_df['lang'] == 'en']
        train_data = make_sequence_data_test(total_test_df)
        train_data = train_data.reset_index()

        return train_data


    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        source = self.df.loc[index, 'text']
        
        source = clean_twitter_text_result(source, self.tt)
        #label = self.df.loc[index, 'label']
        inputs = self.tokenizer.encode_plus(
          source,
          '',
          padding='max_length',
          add_special_tokens=True,
          return_attention_mask=True,
          max_length=self.maxlen,
          truncation=True
          )
        ids = inputs['input_ids']
        #token_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']

        tokens_ids_tensor = torch.tensor(ids,dtype=torch.long)
        attn_mask = torch.tensor(mask, dtype=torch.long)
        
        return tokens_ids_tensor, attn_mask

Build data loaders 

In [15]:
train_set = TweetDataset(data_dir= '/content/drive/MyDrive/NLP_data/t1_train', maxlen = 120,model_name='cardiffnlp/twitter-roberta-base')

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [16]:
dev_set = TweetDataset(data_dir= '/content/drive/MyDrive/NLP_data/t1_dev', maxlen = 120,model_name='cardiffnlp/twitter-roberta-base')

In [28]:
from torch.utils.data import DataLoader
#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size = 64, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 64, num_workers = 2)

print("Done preprocessing training and development data.")

Done preprocessing training and development data.


In [18]:
train_set.df

,index,source_text,problem_num,reply,label
0,0,5. Can regularly rinsing your nose with saline...,0,6. Do vaccines against pneumonia protect you a...,0
1,0,French police chief killed himself after #Char...,1,@Rene_gadeCowboy @Telegraph. Nothing to see he...,1
2,1,French police chief killed himself after #Char...,1,@Telegraph @wmiddelkoop it stinkS !!! These wh...,1
3,2,French police chief killed himself after #Char...,1,@Telegraph And tasked with investigating Charl...,1
4,3,French police chief killed himself after #Char...,1,"FWIW, his 2nd in command comitted suicide in 2...",1
...,...,...,...,...,...
12206,8,Peek behind the curtain of our Save The Americ...,998,@Visit_CostaRica I LOVE SLOTHS!!!!,1
12207,9,Peek behind the curtain of our Save The Americ...,998,Too funny! RT@Visit_CostaRica: Peek behind the...,1
12208,10,Peek behind the curtain of our Save The Americ...,998,@Visit_CostaRica cc @luisassardo,1
12209,0,Donald Trump v Adolf Hitler https://t.co/cBEDe...,999,"Donald Trump isn't a Farcicist, he's just bein...",1


In [19]:
dev_set.df['source_text'][0]

'COVID-19 Fact:\nAre hand dryers effective in killing the new coronavirus?\n\nShare the information with your loved ones and help keep them safe.\n\nSource : WHO\n#weatherbug #weather #knowbefore #wx #istayhomefor #wecan  #corona #quarantinelife #strongertogether #wewillprevail https://t.co/dsPnQLUpMy'

In [20]:

ids = dev_set.__getitem__(10)[0]
dev_set.tokenizer.convert_ids_to_tokens(ids)

['<s>',
 'no',
 'vel',
 'Ġcoron',
 'av',
 'irus',
 'Ġ(',
 'Ġ2019',
 'Ġ-',
 'Ġn',
 'c',
 'ov',
 'Ġ)',
 'Ġadvice',
 'Ġfor',
 'Ġthe',
 'Ġpublic',
 'Ġ:',
 'Ġmyth',
 'Ġbust',
 'ers',
 'Ġcor',
 'ona',
 'Ġvirus',
 'Ġare',
 'Ġantibiotics',
 'Ġeffective',
 'Ġin',
 'Ġpreventing',
 'Ġand',
 'Ġtreating',
 'Ġthe',
 'Ġnew',
 'Ġcoron',
 'av',
 'irus',
 'Ġ?',
 'Ġcourt',
 'sey',
 'Ġ-',
 'Ġ@',
 'user',
 'Ġhttp',
 '</s>',
 '</s>',
 'no',
 'vel',
 'Ġcoron',
 'av',
 'irus',
 'Ġ(',
 'Ġ2019',
 'Ġ-',
 'Ġn',
 'c',
 'ov',
 'Ġ)',
 'Ġadvice',
 'Ġfor',
 'Ġthe',
 'Ġpublic',
 'Ġ:',
 'Ġmyth',
 'Ġbust',
 'ers',
 'Ġcor',
 'ona',
 'Ġvirus',
 'Ġare',
 'Ġthere',
 'Ġany',
 'Ġspecific',
 'Ġmedicines',
 'Ġto',
 'Ġprevent',
 'Ġor',
 'Ġtreat',
 'Ġthe',
 'Ġnew',
 'Ġcoron',
 'av',
 'irus',
 'Ġ?',
 'Ġcourt',
 'sey',
 'Ġ-',
 'Ġ@',
 'user',
 'Ġhttp',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pa

In [21]:
dev_set.df['source_text'][10]

'Novel Coronavirus (2019-nCoV) advice for the public: #MythBusters #Coronavirus\n\nAre antibiotics effective in preventing and treating the new coronavirus?\n\nCourtsey - @WHO https://t.co/58UuU7Sr4p'

Define and train model

In [22]:
import torch
torch.cuda.empty_cache()
import torch.nn as nn
from transformers import BertModel
from transformers import AutoTokenizer, AutoModel

#tokenizer = AutoTokenizer.from_pretrained("tristantristantristan/rumor")

class RumorModel(nn.Module):
  def __init__(self,model_name):
    super(RumorModel, self).__init__()
    #self.MODEL = 'cardiffnlp/twitter-roberta-base-sentiment'
    #self.bert_model = AutoModelForSequenceClassification.from_pretrained(self.MODEL)
    self.MODEL = model_name
    self.bert_model = AutoModel.from_pretrained(self.MODEL)
    #self.bert_model = AutoModel.from_pretrained('cardiffnlp/twitter-roberta-base-2021-124m')
    self.dropout = nn.Dropout(p=0.3)
    self.cls_layer = nn.Linear(768,1)

  def forward(self,ids,mask):
    outputs = self.bert_model(ids,attention_mask=mask, return_dict=True)
    pooled_output = outputs.pooler_output
    
    #pooled_output = torch.cat(tuple([hidden_states[:,i] for i in [-4, -3, -2, -1]]), dim=-1)
    #pooled_output = self.dropout(pooled_output)
    # classifier of course has to be 4 * hidden_dim, because we concat 4 layers
    
    #logits = self.cls_layer(pooled_output)

    #last = outputs.last_hidden_state
    #cls_rep = last[:,0]

    cls_rep = self.dropout(pooled_output)
    logits = self.cls_layer(cls_rep)
    return logits



In [23]:
gpu = 0 #gpu ID

print("Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...")
net=RumorModel('cardiffnlp/twitter-roberta-base')

net.cuda(gpu) #Enable gpu support for the model
print("Done creating the sentiment classifier.")

Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done creating the sentiment classifier.


In [24]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.AdamW(net.parameters(), lr = 2e-5)

In [25]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu,lr):

    best_acc = 0
    best_ep = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu),  labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)
            #logits = net(seq, attn_masks, type_ids)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()
        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            try:
              os.remove('/content/drive/MyDrive/NLP_data/sstcls_{}_lr_{}_val_{}.dat'.format(best_ep,lr,round(dev_loss,3)))
            except:
              pass
            best_ep = ep
            torch.save(net.state_dict(), '/content/drive/MyDrive/NLP_data/sstcls_{}_lr_{}_val_{}.dat'.format(ep,lr,round(dev_loss,3)))
              
            

In [26]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [27]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
num_epoch = 8

#fine-tune the model
opti_list = [1e-3,1e-4,5e-5,1e-5]
for lr in opti_list:
  opti = optim.AdamW(net.parameters(),
                    lr = 5e-5,
                    eps = 1e-8,
                    weight_decay = 1e-2
                    )
  #pti = optim.AdamW(net.parameters(), lr = lr)
  train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu,lr)

Iteration 0 of epoch 0 complete. Loss: 0.6976836919784546; Accuracy: 0.5; Time taken (s): 1.130561113357544


Build test set and predict

In [ ]:
test_set = TestDataset(data_dir='/content/drive/MyDrive/NLP_data/project-data',maxlen=120)

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
miss = [69, 6, 459, 313, 380]

In [ ]:
a = test_set.df.loc[test_set.df['problem_num'].isin(miss)]
a

,index,text,problem_num
12,0,เผยกบฏยูเครนอาจเข้าใจผิดคิดว่า #MH17 เป็นเครื่...,6
557,0,هل المضادات الحيوية فعَّالة في الوقاية من فيرو...,69
1904,0,@DomicelaH Cold weather and snow CANNOT kill t...,313
2388,0,Se ha detectado SARS-CoV-2 en gatos y perros d...,380
2916,0,هل تنشر الحيوانات الأليفة بالمنزل فيروس كورونا...,459


In [ ]:
from torch.utils.data import DataLoader
test_loader = DataLoader(test_set, batch_size=32, num_workers=2)

In [ ]:
test_set.__getitem__(1)

(tensor([    0,  1039, 12105,   939,  4157,     7,   489,   584,    24,  2156,
            53, 21314,    40, 12956,  4636,   111,    19,    50,   396,   111,
            10,  6793,   479, 20343,    16,    10, 37968,   118,  1555,  2054,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
test_set.df['text'].values[4]

'@_truthpolitics We should absolutely blame the politicians and whoever else is involved in not holding police offic… https://t.co/cuYdXOCQmG @klarth Yeah. I never said we shouldn’t hold them accountable, nor did I insinuate that. We obviously need to do th… https://t.co/r7D7rvV3mY '

load trained model

In [ ]:
gpu = 0
path = '/content/drive/MyDrive/NLP_data/sstcls_2.dat'
model = RumorModel()
model.load_state_dict(torch.load(path))
model.cuda(gpu)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-mar2022 were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-mar2022 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

RumorModel(
  (bert_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [ ]:
def predict(model, dataloader, gpu):
    model.eval()

    #mean_acc, mean_loss = 0, 0
    count = 0
    prob_list = []
    predicted_label_list = []
    with torch.no_grad():
        for seq, attn_masks in dataloader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = model(seq, attn_masks)
            probs = torch.sigmoid(logits.unsqueeze(-1))
            soft_probs = (probs > 0.5).long()

            predicted_label_list.append(soft_probs.squeeze().cpu().detach().numpy())

#            acc = (soft_probs.squeeze() == labels)
 #           prob_list.append(soft_probs)
            #predicted_label_list.append(soft_probs.squeeze())
            #mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            #mean_acc += get_accuracy_from_logits(logits, labels)
            #count += 1

    return prob_list, predicted_label_list

In [ ]:
gpu = 0
prob_list, predicted_label_list = predict(model, test_loader,gpu )

In [ ]:
predicted_label_list

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 

In [ ]:
import numpy as np
def output_label(test_df, label_list):
  output = label_list[0]
  for tensor in label_list[1:]:
    #temp_arr = tensor.cpu().detach().numpy()
    #print(temp_arr)
    output = np.concatenate((output, tensor))
  test_df['label'] = output
  #output_df = test_df.loc[['problem_num','label']]
  return test_df

In [ ]:
output_df = output_label(test_set.df,predicted_label_list)

In [ ]:
def majority_vote(df:pd.DataFrame):
  groupped = df.groupby(['problem_num'])
  big_dict = []
  from collections import Counter
  for pro, small in groupped:
    small_dict = {
        'Id':int(pro)
    }
    sb = Counter(small['label'])
    t = sb[1]
    f = sb[0]
    if t > f:
      small_dict['Predicted'] = 1
    else:
      small_dict['Predicted'] = 0
    big_dict.append(small_dict)
  output = pd.DataFrame(big_dict)
  return output


In [ ]:
out = majority_vote(output_df)
out

,Id,Predicted
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
553,553,0
554,554,0
555,555,1
556,556,0


In [ ]:
out.to_csv('split_hashtag.csv',index=False)